In [13]:
from mandala._next.imports import *
from mandala._next.common_imports import *

In [14]:
@op # a function with a wild input signature
def add(x, *args, y: int = 1, **kwargs):
    # just sum everything
    return x + sum(args) + y + sum(kwargs.values())

@op # a function with variable number of outputs
def factorize(n: int) -> Tuple[int, ...]:
    factors = []
    for i in range(2, n + 1):
        while n % i == 0:
            factors.append(i)
            n //= i
    return tuple(factors)

@op # function with list input and dict output
def get_stats(nums: MList[int]) -> MDict[str, float]:
    return {
        'avg': float(sum(nums) / len(nums)),
        'sum': float(sum(nums)),
    }

In [15]:
storage = Storage()

In [17]:
with storage:
    add(1)
    add(1, 2, 3, 4, )
    add(1, 2, 3, 4, y=5)
    add(1, 2, 3, 4, y=5, z=6)


In [18]:
cf = storage.cf(add)
cf

ComputationFrame with 7 variable(s) (10 unique refs), 1 operation(s) (4 unique calls)
Computational graph:
    output_0 = add(args_1=args_1, z=z, x=x, args_2=args_2, y=y, args_0=args_0)

In [ ]:
@op
def f(x):
    return x ** 2 + 1

@op
def g(x, y):
    return x + y

@op
def h(x):
    return x + 1


In [ ]:

storage = Storage(deps_path='__main__')

# memoizing context
with storage:
    for x in range(3):
        y = f(x)

y

In [ ]:
with storage:
    for x in range(5):
        y = f(x)
        z = g(y, h(x))
    
    for y in range(15, 20):
        z = g(y, h(x))

In [ ]:
cf = storage.cf(g); cf

In [ ]:
exp_cf = cf.expand(); exp_cf

In [ ]:
exp_cf.get_df()

In [ ]:
v = storage.get_versioner()

In [ ]:
with storage:
    for x in range(5):
        y = f(x)
        if storage.unwrap(y) > 5:
            z = g(x, y)

In [ ]:
storage.cf(f).\
  expand().\
    eval()

In [ ]:
df.rename(columns={'output_0_0': 'output_1'})

In [ ]:
cf = storage.cf(g)
cf.expand(inplace=True)
cf.eval()

In [ ]:
cf.delete_calls()

In [ ]:
@op
def avg_items(xs: MList[int]) -> float:
    return sum(xs) / len(xs)

@op
def get_xs(n: int) -> MList[int]:
    return list(range(n))

storage = Storage()

with storage:
    xs = get_xs(10)
    for i in range(2, 10, 2):
        avg = avg_items(xs[:i])

In [ ]:
cf = storage.cf(avg_items).expand()
cf.eval()

In [ ]:
storage = Storage()

@op
def inc(eggs):
    print(f'Executing inc with eggs={eggs}')
    return eggs + 1

with storage:
    spam = inc(eggs=42)

@op
def make_breakfast(spam, eggs):
    print(f'Executing make_breakfast with spam={spam} and eggs={eggs}')
    return spam + eggs

with storage:
    eggs = 42
    spam = inc(eggs=eggs)
    more_spam = inc(eggs=spam)
    breakfast = make_breakfast(spam=spam, eggs=eggs)
    breakfast_2 = make_breakfast(spam=more_spam, eggs=eggs)


cf = storage.cf(make_breakfast)
cf

cf = cf.expand()
cf.eval()

In [ ]:
storage = Storage()

### a few simple ops covering all cases we care about
@op
def f(x: int) -> int:
    return x + 1

@op
def g(x: int, y: int) -> Tuple[int, int]: # multiple inputs and outputs
    return x + 2, y ** 2

@op 
def inc_list(things: MList[int]) -> MList[int]: # lists as inputs and outputs
    return [x + 1 for x in things]

@op
def inc_dict(things: MDict[str, int]) -> MDict[str, int]: # dicts as inputs and outputs
    return {k: v + 1 for k, v in things.items()}

@op
def fancy_signature(x: int, *args, y: bool = False, **kwargs) -> Tuple[int, List[int], MList[int]]: # fancy signatures
    return x, [int(a) for a in args], [int(v) for v in kwargs.values()]

In [ ]:
### simple workflow
with storage:
    x = f(1)
    y, t = g(x, 2)
    z = inc_list([1, 2])
    z = inc_list([x, y])
    z = inc_list([1, x])
    w = inc_dict({'x': x, 'y': y})
    a = fancy_signature(1, 2, x, y=True, z=4, w=5)